In [3]:
import sys
import os

In [4]:
sys.path.append(os.path.abspath('../execute'))

In [20]:
import json
import parcels
from datetime import timedelta
import kernels
import logging
import time
from importlib import reload  # Python 3.4+


In [21]:
import execute_PARCELS as exP

In [22]:
reload(exP)
reload(kernels)

<module 'kernels' from '/home/colin_beyers/lagrangian_drifts/execute/kernels.py'>

In [23]:
config = exP.load_config(config_path='../execute/config.json')

2024-09-27 12:45:57,052 - INFO - Loading configuration from ../execute/config.json


In [24]:
config

{'pathname': '/home/colin_beyers/data/cllc-U,V,mag.nc4',
 'variables': {'U': 'U', 'V': 'V'},
 'dimensions': {'lat': 'lat', 'lon': 'lon', 'time': 'time'},
 'particles': {'lon': [-135], 'lat': [40.0]},
 'kernels': ['AdvectionRK4', 'TotalDistance', 'BoundaryCheck'],
 'output': {'directory': '../output',
  'file_name': 'cllc-total_eulerian-TEST.zarr'},
 'runtime': {'days': 5, 'dt_minutes': 5},
 'output_interval_hours': 6}

In [25]:
kernel_list = exP.load_kernels(config.get("kernels", []), parcels, kernels)

2024-09-27 12:45:57,641 - INFO - Loaded kernels: ['AdvectionRK4', 'TotalDistance', 'BoundaryCheck']


In [26]:
kernel_list

[<function parcels.application_kernels.advection.AdvectionRK4(particle, fieldset, time)>,
 <function kernels.TotalDistance(particle, fieldset, time)>,
 <function kernels.BoundaryCheck(particle, fieldset, time)>]

In [27]:
extra_vars = exP.collect_extra_vars(kernel_list)

2024-09-27 12:45:58,597 - INFO - Collected extra variables for kernels: [PVar<distance|<class 'numpy.float32'>>, PVar<prev_lon|<class 'numpy.float32'>>, PVar<prev_lat|<class 'numpy.float32'>>, 'lat_min', 'lat_max', 'lon_min', 'lon_max']


In [31]:
type(extra_vars[0])

parcels.particle.Variable

In [33]:
for extra_var in extra_vars:
    if type(extra_var) != parcels.particle.Variable:
        print(extra_var)

lat_min
lat_max
lon_min
lon_max


In [58]:
fieldset = exe.create_fieldset(config)

2024-09-26 15:15:51,815 - INFO - Creating FieldSet with filenames: {'U': '/home/colin_beyers/data/cllc-U,V,mag.nc4', 'V': '/home/colin_beyers/data/cllc-U,V,mag.nc4'}


In [59]:
for extra_var in extra_vars:
    fieldset.add_constant(extra_var.name, extra_var)

In [48]:
extra_var

PVar<lat_max|<class 'numpy.float32'>>

In [51]:
parcels.Variable?

Init signature:
parcels.Variable(
    name,
    dtype=<class 'numpy.float32'>,
    initial=0,
    to_write=True,
)
Docstring:     
Descriptor class that delegates data access to particle data.

Parameters
----------
name : str
    Variable name as used within kernels
dtype :
    Data type (numpy.dtype) of the variable
initial :
    Initial value of the variable. Note that this can also be a Field object,
    which will then be sampled at the location of the particle
to_write : bool, 'once', optional
    Boolean or 'once'. Controls whether Variable is written to NetCDF file.
    If to_write = 'once', the variable will be written as a time-independent 1D array
File:           ~/miniconda3/envs/parcels_working/lib/python3.12/site-packages/parcels/particle.py
Type:           type
Subclasses:     

In [57]:
lon_min = fieldset.U.grid.lon[0]
lon_max = fieldset.U.grid.lon[-1]
lat_min = fieldset.U.grid.lat[0]
lat_max = fieldset.U.grid.lat[-1]

In [ ]:
fieldset.add_constant("lon_min", lon_min)
fieldset.add_constant("lon_max", lon_max)
fieldset.add_constant("lat_min", lat_min)
fieldset.add_constant("lat_max", lat_max)